<a href="https://colab.research.google.com/github/busyhound/ML-2/blob/master/ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets

from sklearn.model_selection import train_test_split
iris = datasets.load_iris()
X, Y_in = iris.data, iris.target
#print(len(y))

#x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.01,random_state=123)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1)

In [ ]:
def NeuralNetwork(X_train, Y_train,epochs=10, nodes=[], lr=0.15):
    hidden_layers = len(nodes) - 1
    layers=len(nodes)
    weights=[]
    for i in range(1, layers):
      w = [[np.random.uniform(-1, 1) for k in range(nodes[i-1] + 1)]
           for j in range(nodes[i])]
      weights.append(np.matrix(w))
 
    for epoch in range(1, epochs+1):

      layers = len(weights)
      for i in range(len(X)):

        x, y = X[i], Y[i]
        x = np.matrix(np.append(1, x)) # Augment feature vector
        
        activations = ForwardPropagation(x, weights, layers)
        weights = BackPropagation(y, activations, weights, layers)
      if(epoch % 20 == 0):
        print("Epoch {}".format(epoch))
        print("Training Accuracy:{}".format(Accuracy(X_train, Y_train, weights)))
        print("Test Accuracy:{}".format(Accuracy(X_test, Y_test, weights)))
      #  weights = Train(X_train, Y_train, lr, weights)

        
            
    return weights


In [ ]:
def oneHotVector(n):
  b=[]
  #a=int(input())
  a=3 # number of class
  for i in range(a):
    if(i==n):
      b.append(1.0)
    else:
      b.append(0.0)
  return b

In [ ]:
def softmax(x):
    x=np.array(x)
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()


In [ ]:
def outputClass(softmaxValue):
  softmaxValue=np.array(softmaxValue)
 # maxpos = softmaxValue.index(max(softmaxValue))
  maxpos= np.argmax(softmaxValue)
  #print("max")
  #print(maxpos)
  for i in range(len(softmaxValue)):
    if(i==maxpos):
      softmaxValue[i]=maxpos
    else:
      softmaxValue[i]=0
  return softmaxValue.tolist()


In [ ]:
def final_output(softmax):
  softmaxValue=np.array(softmax)
 # maxpos = softmaxValue.index(max(softmaxValue))
  maxpos= np.argmax(softmaxValue)
  return maxpos


In [ ]:
def dot(x, y):
    """Dot product as sum of list comprehension doing element-wise multiplication"""
    return sum(x_i*y_i for x_i, y_i in zip(x, y))

def cross_product(u,v):  
    dim = len(u)
    s = []
    for i in range(dim):
        if i == 0:
            j,k = 1,2
            s.append(u[j]*v[k] - u[k]*v[j])
        elif i == 1:
            j,k = 2,0
            s.append(u[j]*v[k] - u[k]*v[j])
        else:
            j,k = 0,1
            s.append(u[j]*v[k] - u[k]*v[j])
    return s


In [ ]:
def ForwardPropagation(x, weights, layers):
    activations, layer_input = [x], x
    for j in range(layers):
        activation = Sigmoid(np.dot(layer_input, weights[j].T))
        #print("activation",activation)
        #input()
        activations.append(activation)
        layer_input = np.append(1, activation) # Augment with bias
    #print("activations----------------",activations)
    #input()
    return activations

In [ ]:
def BackPropagation(y, activations, weights, layers):
    outputFinal = activations[-1]
    error = np.matrix(y - outputFinal) # Error at output
    
    for j in range(layers, 0, -1):
        currActivation = activations[j]
       #print("currActivation",currActivation)
        #input()
        
        if(j > 1):
            # Augment previous activation
            prevActivation = np.append(1, activations[j-1])
        else:
            # First hidden layer, prevActivation is input (without bias)
            prevActivation = activations[0]
        
        delta = np.multiply(error, SigmoidDerivative(currActivation))
        weights[j-1] += lr * np.multiply(delta.T, prevActivation)

        w = np.delete(weights[j-1], [0], axis=1) # Remove bias from weights
        error = np.dot(delta, w) # Calculate error for current layer
    
    return weights

In [ ]:
def Sigmoid(x):
    return 1 / (1 + np.exp(-x))

def SigmoidDerivative(x):
    return np.multiply(x, 1-x)

In [ ]:
def Predict(item, weights):
    layers = len(weights)
    item = np.append(1, item) # Augment feature vector
    
    ##_Forward Propagation_##
    activations = ForwardPropagation(item, weights, layers)
    
    outputFinal = activations[-1].A1
    index = FindMaxActivation(outputFinal)

    # Initialize prediction vector to zeros
    y = [0 for i in range(len(outputFinal))]
    y[index] = 1  # Set guessed class to 1

    return y # Return prediction vector


def FindMaxActivation(output):
    """Find max activation in output"""
    m, index = output[0], 0
    for i in range(1, len(output)):
        if(output[i] > m):
            m, index = output[i], i
    
    return index

In [ ]:
def Accuracy(X, Y, weights):
    """Run set through network, find overall accuracy"""
    correct = 0
    #print("rr3")
    for i in range(len(X)):
        #print("rr4")
        x, y = X[i], list(Y[i])
      #  print(y)
        #input()
       # print("rr1")
        guess = Predict(x, weights)
        #print("rr2")

        if(y == guess):
            # Guessed correctly
            correct += 1

    return correct / len(X)

In [ ]:
inputLayer=4
outputLayer=3
First_hiddenLayer=5
layers=[inputLayer,First_hiddenLayer,outputLayer]
lr, epochs = 0.15, 100


weights = NeuralNetwork(X_train, Y_train, epochs=epochs, nodes=layers, lr=lr);


Epoch 20
Training Accuracy:0.6574074074074074
Test Accuracy:0.7
Epoch 40
Training Accuracy:0.7777777777777778
Test Accuracy:0.7333333333333333
Epoch 60
Training Accuracy:0.8611111111111112
Test Accuracy:0.8666666666666667
Epoch 80
Training Accuracy:0.8611111111111112
Test Accuracy:0.9666666666666667
Epoch 100
Training Accuracy:0.8703703703703703
Test Accuracy:0.9666666666666667


In [ ]:
print(x_test)
X_train

[[6.3 2.5 4.9 1.5]
 [6.8 3.  5.5 2.1]
 [6.4 2.8 5.6 2.2]
 [5.6 3.  4.1 1.3]
 [4.9 3.6 1.4 0.1]
 [6.  3.  4.8 1.8]
 [6.3 2.3 4.4 1.3]
 [4.4 3.2 1.3 0.2]
 [4.4 2.9 1.4 0.2]
 [5.5 2.6 4.4 1.2]
 [6.9 3.1 5.1 2.3]
 [5.5 4.2 1.4 0.2]
 [5.2 2.7 3.9 1.4]
 [6.5 3.  5.5 1.8]
 [7.7 3.  6.1 2.3]
 [6.5 3.  5.8 2.2]
 [5.5 3.5 1.3 0.2]
 [4.3 3.  1.1 0.1]
 [6.1 2.9 4.7 1.4]
 [4.8 3.  1.4 0.3]
 [5.2 3.4 1.4 0.2]
 [6.3 2.8 5.1 1.5]
 [4.8 3.4 1.9 0.2]
 [6.1 3.  4.9 1.8]
 [5.1 3.8 1.6 0.2]
 [5.4 3.4 1.7 0.2]
 [5.4 3.4 1.5 0.4]
 [5.6 2.8 4.9 2. ]
 [7.7 3.8 6.7 2.2]
 [5.  3.6 1.4 0.2]]


array([[6. , 2.9, 4.5, 1.5],
       [5.2, 3.5, 1.5, 0.2],
       [6.1, 2.8, 4.7, 1.2],
       [5.7, 2.6, 3.5, 1. ],
       [4.6, 3.4, 1.4, 0.3],
       [4.7, 3.2, 1.3, 0.2],
       [5.6, 2.5, 3.9, 1.1],
       [5.7, 4.4, 1.5, 0.4],
       [4.8, 3.1, 1.6, 0.2],
       [5.8, 2.6, 4. , 1.2],
       [5.8, 2.7, 5.1, 1.9],
       [4.7, 3.2, 1.6, 0.2],
       [5.5, 2.3, 4. , 1.3],
       [6.8, 3.2, 5.9, 2.3],
       [5. , 3.4, 1.6, 0.4],
       [5.9, 3.2, 4.8, 1.8],
       [6.5, 2.8, 4.6, 1.5],
       [6.7, 3. , 5. , 1.7],
       [4.6, 3.1, 1.5, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [6.3, 2.7, 4.9, 1.8],
       [6.5, 3. , 5.2, 2. ],
       [5.1, 2.5, 3. , 1.1],
       [6.2, 2.9, 4.3, 1.3],
       [6.4, 3.1, 5.5, 1.8],
       [7.2, 3.6, 6.1, 2.5],
       [5.5, 2.4, 3.8, 1.1],
       [6.3, 3.3, 4.7, 1.6],
       [5.3, 3.7, 1.5, 0.2],
       [4.6, 3.6, 1. , 0.2],
       [5.5, 2.5, 4. , 1.3],
       [6.7, 3. , 5.2, 2.3],
       [6.7, 3.1, 5.6, 2.4],
       [7.9, 3.8, 6.4, 2. ],
       [6.4, 2

In [ ]:
#weights = NeuralNetwork(X_train, Y_train, X_val, Y_val, epochs=epochs, nodes=layers, lr=lr);
#accu=Accuracy(x_test,y_test,weights)
#for i in weights:
  #print("Training Accuracy:{}".format(Accuracy(x_test, y_test, i)))
  #print(i)

#print("Training Accuracy:{}".format(Accuracy(x_test, y_test, weights)))
#len(weights)
print(y_test)

[1 2 2 1 0 2 1 0 0 1 2 0 1 2 2 2 0 0 1 0 0 2 0 2 0 0 0 2 2 0]
